In [ ]:
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
with open('../trial03/peer_query_count.json', 'r') as f:
    peer_query_count = json.load(f)
result = peer_query_count['data']['result']
peers = list({r['metric']['self'] for r in result})
print(peers)

In [ ]:
def get_success_ts(result, self, other, endpoint, query_type):
    for ts in result:
        metric = ts['metric']
        if metric['endpoint'] == endpoint and metric['self'] == self and metric['other'] == other and \
            metric['outcome'] == 'SUCCESS' and metric['query_type'] == query_type:
            times, values = [], []
            for time_value in ts['values']:
                times.append(time_value[0])
                values.append(int(time_value[1]))
            return pd.Series(values, pd.to_datetime(times, unit='s'))

In [ ]:
endpoint='Find'
fig, axes = plt.subplots(ncols=2, nrows=4, sharey=True, sharex=True, figsize=(15,15))

for i, self in enumerate(peers[8:16]):
    diffs = {}
    for j, other in enumerate(peers):
        rq_values = get_success_ts(result, self, other, endpoint, query_type='REQUEST')
        rp_values = get_success_ts(result, self, other, endpoint, query_type='RESPONSE')
        diffs[other] = rq_values / (rq_values + rp_values)

    x, y = int(i / 2), int(i % 2)
    axes[x, y].plot(pd.DataFrame(diffs))